# Imports

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import cv2
from skimage import io
import csv
import numpy as np
from pathlib import Path
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from feature_extractor import FeatureExtractor
from zipfile import ZipFile
import os
from sklearn.decomposition import PCA
import tensorflow as tf

# Features extraction

In [ ]:
fe = FeatureExtractor()

log = open("log.txt","a")
features_list = [i for i in range(4096)]
features_list.append("image_url")
for k in range(0,10):
    df = pd.read_csv(f"open-images-dataset-train{k}.tsv","\t",header=0,names = ["URL",'ID',"T"])
    for i in tqdm(range(0,100)):
        skipped = 0
        f = open(f'vgg16_features/features_0{k}{i}.csv',"w")
        writer = csv.writer(f)
        writer.writerow(features_list)
        for j in tqdm(range(i*10000,(i+1)*10000)):
            try:
                url = df.iloc[j]["URL"]
                im = io.imread(df.iloc[j]["URL"])
                features = fe.extract(img=im)
                features = features.tolist()
                features.append(url)
                writer.writerow(features)
            except Exception as e:
                skipped+=1
        f.close()
        log.write(f"Batch {i} contains {10000-skipped} image feature\n")
    log.close()

# Clean data with tags

In [2]:
def extract_id(row):
    return str(row["OriginalURL"].split("/")[-1].split("_")[0])

In [29]:
first = pd.read_csv(f"open-images-dataset{0}_withtags.csv")
first["id"] = first.apply(lambda row: extract_id(row), axis=1)
first["id"] = first["id"].astype("str")
for i in range(1,10):
    second = pd.read_csv(f"open-images-dataset{i}_withtags.csv")
    second["id"] = second.apply(lambda row: extract_id(row), axis=1)
    second["id"] = second["id"].astype("str")
    res = pd.concat([first,second])
    first = res
res.to_csv("images_meta.csv",index=False)

C:\Users\gtoun\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
clean_df = final.groupby(['OriginalURL','Label','Author','Title',"Tags","id"])['ImageID'].apply(', '.join).reset_index()